In [1]:
!pip install yadisk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 864.6 kB/s eta 0:00:00


In [2]:
import yadisk
import os
from tqdm import tqdm
import sys
import openpyxl
import pandas as pd
import re
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt

In [3]:
#убираем ворнинги
import warnings
warnings.filterwarnings("ignore")

In [4]:
#устанавливаем степень точности отображения чисел
pd.set_option('display.precision', 2)

### Скачиваем файлы с яндекс-диска.

In [5]:
app_id = '...'
secret_id = '...'

In [9]:
y = yadisk.YaDisk(app_id, secret_id)
url = y.get_code_url()
print("Go to the following url: %s" % url)
code = input("Enter the confirmation code: ")

try:
    response = y.get_token(code)
except yadisk.exceptions.BadRequestError:
    print("Bad code")
    sys.exit(1)
y.token = response.access_token
if y.check_token():
    print("Готово, подконнектились.")
else:
    print("Что-то не так.")

Go to the following url: https://oauth.yandex.ru/authorize?response_type=code&client_id=734e4c9f1eab43f8b6526a157f6a7ee6&display=popup&force_confirm=yes
Enter the confirmation code: 7048810
Готово, подконнектились.


In [10]:
#собираем список файлов, которые надо скачать
list_of_files = []
for item in list(y.listdir('file_processor')):
    if item['path'].endswith('.xlsx') or item['path'].endswith('.json'):
        list_of_files.append(item['path'])

In [11]:
#создаём папку, куда их положим, и заходим в неё
load_path = '/content/lab'
if not os.path.exists(load_path):
    os.mkdir(load_path)
os.chdir(load_path)

In [12]:
#скачиваем файлы с яндекс-диска
for file in tqdm(list_of_files):
  if file.split('/')[-1] not in os.listdir():
    y.download(file.split(':')[1], file.split('/')[-1])

100%|██████████| 148/148 [04:34<00:00,  1.85s/it]


### Разделяем файлы на 4 списка, из которых потом по-отдельности будем собирать таблицы.

In [13]:
#функции чтобы посмотреть, что в первых строчках таблиц

#смотрим, есть ли в первых строках нужный текст
def check_string_in_first_rows(worksheet, target_string):
    for row in worksheet.iter_rows(min_row=0, max_row=5):
        for cell in row:
            if cell.value and target_string.upper() in str(cell.value).upper():
                return True
    return False

#возвращаем список таблиц, в которых есть нужный текст
def check_first_rows(list, target_string):
    list_of_tables = []
    for name in list:
      file_path = f'/content/lab/{name}'
      workbook = openpyxl.load_workbook(file_path, read_only=True)
      worksheet = workbook.worksheets[0]

      if check_string_in_first_rows(worksheet, target_string):
          list_of_tables.append(file_path.split('/')[-1])

    return list_of_tables

In [14]:
#делим на списки по названию
sop_list, zno_list = [], []
for name in os.listdir():
  if 'состояние_онко_помощи' in name.lower():
    sop_list.append(name)
  if 'злокачественные_новообразования' in name.lower():
      zno_list.append(name)

In [15]:
#делим на списки по содержанию внутри файла
target_strings_sop = ['лечении злокачественных', 'контингенте']
target_strings_zno = ['заболеваемость населения', 'смертность населения']

sop_treatment = check_first_rows(sop_list, target_strings_sop[0])
sop_population = check_first_rows(sop_list, target_strings_sop[1])
zno_incidence = check_first_rows(zno_list, target_strings_zno[0])
zno_mortality = check_first_rows(zno_list, target_strings_zno[1])

### Все нужные функции

In [16]:
#функция, чтобы раздербанить первую клетку таблицы Состояние_онко_помощи на данные для дополнительных колонок:
#год, название заболевания, код заболевания, номер таблицы
def sop_new_cols(table):

    values = {}

    #открываем файл
    workbook = openpyxl.load_workbook(table, read_only=True)
    worksheet = workbook.worksheets[0]

    #читаем первую клетку
    first_cell = worksheet.cell(row=1, column=1).value

    #ищем год
    year_match = re.search(r'\b\d{4}\b', first_cell)
    year = year_match.group(0) if year_match else None
    values.update({'year': year})

    #ищем локализацию заболевания
    name_match = re.search(r'\n(.*?)(\(C\d+|\(С\d+)', first_cell)
    name = name_match.group(1).strip() if name_match else None
    values.update({'localization': name})

    #ищем код заболевания
    code_match = re.search(r'\((C[^)]+|С[^)]+)\)', first_cell)
    code = code_match.group(1) if code_match else None
    values.update({'icd_code': code})

    #ищем номер таблицы
    table_match = re.search(r'Таблица (\d+)', first_cell)
    table = table_match.group(1) if table_match else table.split('_')[2]
    values.update({'table': table})

    return values

In [17]:
#функция, чтобы раздербанить первые клетки таблицы Злокачественные_новообразования на данные для дополнительных колонок:
#год, название заболевания, код заболевания, номер таблицы
def zno_new_cols(table):

    values = {}

    workbook = openpyxl.load_workbook(table, read_only=True)
    worksheet = workbook.worksheets[0]

    if worksheet['B3'].value is None:
      year_match = re.search(r'\d{4}$', worksheet['A3'].value)
      year = year_match.group(0).strip() if year_match else None
    else:
      year = worksheet['B3'].value
    values.update({'year': year})

    if worksheet['B4'].value is None:
      name_match = re.search(r': (.*?)(\(C\d+|\(С\d+)', worksheet['A4'].value)
    else:
      name_match = re.search(r'(.*?)(\(C\d+|\(С\d+)', worksheet['B4'].value)
    name = name_match.group(1).strip() if name_match else worksheet['B4'].value.strip()
    values.update({'localization': name})

    if worksheet['B4'].value is None:
      code_match = re.search(r'\((C[^)]+|С[^)]+)\)', worksheet['A4'].value)
    else:
      code_match = re.search(r'\((C[^)]+|С[^)]+)\)', worksheet['B4'].value)
    code = code_match.group(1) if code_match else worksheet['B4'].value.strip()
    values.update({'icd_code': code})

    table_match = re.search(r'Таблица (\d+)', worksheet['A1'].value)
    table = table_match.group(1) if table_match else table.split('_')[2]
    values.update({'table': table})

    return values

In [18]:
#функция, чтобы склеить мультииндекс столбцов и подправить по мелочи
def column_names_change(df):
  df.columns = df.columns.map(lambda x: ' '.join(map(str, x))) \
                                          .str.replace('\n', ' ') \
                                          .str.replace(r'Unnamed: \d+_level_\d+|- |\.\d$', '', regex=True) \
                                          .str.strip()
  df.columns = [' '.join(s.split()).lower() for s in df.columns]
  df = df.rename(columns={df.columns[0]: 'республика, край, область'})
  return df.columns

In [19]:
#функция, убирающая строки с аггрегированными данными
def remove_aggregated_lines(df):
  df = df[~df.iloc[:, 0].str.contains('россия', case=False) & \
          ~df.iloc[:, 0].str.contains('ФО')] \
          .reset_index(drop=True)
  return df

### Собираем таблицу по данным из таблиц Состояние онко помощи (лечение).

In [20]:
#читаем первую таблицу, чтобы было к чему пришивать остальные
data_sop_treatment = pd.read_excel(sop_treatment[0], header=[1,2])

#создаём новые колонки с годом, заболеванием, кодом заболевания, номером таблицы
new_cols = sop_new_cols(sop_treatment[0])
for key, value in new_cols.items():
    data_sop_treatment[key] = value

In [21]:
#собираем словарик: к какому федеральному округу какая область относится
fo_dict = {}
current_fo = 0

for item in data_sop_treatment.iloc[:, 0]:
  if item.endswith('ФО'):
    current_fo = item
  else:
    fo_dict.update({item: current_fo})

In [22]:
#читаем и конкатенируем все остальные из списка sop_treatment
for file in sop_treatment[1:]:
    df = pd.read_excel(file, header=[1,2])
    new_cols = sop_new_cols(file)
    for key, value in new_cols.items():
        df[key] = value
    data_sop_treatment = pd.concat([data_sop_treatment, df])

In [23]:
#поправляем колонки
data_sop_treatment.columns = column_names_change(data_sop_treatment)

In [24]:
#убираем аггрегированные строки (по стране и по ФО)
data_sop_treatment = remove_aggregated_lines(data_sop_treatment)

In [25]:
#добавляем колонку с федеральным округом для каждой записи
data_sop_treatment['federal_district'] = data_sop_treatment.iloc[:, 0].map(fo_dict)

In [26]:
#подправляем типы
data_sop_treatment['year'] = data_sop_treatment['year'].astype(int)
data_sop_treatment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 15 columns):
 #   Column                                                                                              Non-Null Count  Dtype  
---  ------                                                                                              --------------  -----  
 0   республика, край, область                                                                           2202 non-null   object 
 1   число зно, выявленных в отчетном году, радикальное лечение которых закончено в отчетном году        2202 non-null   int64  
 2   число зно, выявленных в отчетном году, радикальное лечение которых % от впервые выявленных          2202 non-null   float64
 3   число зно, выявленных в отчетном году, радикальное лечение которых будет продолжено (не закончено)  2202 non-null   int64  
 4   число зно, выявленных в отчетном году, радикальное лечение которых % от впервые выявленных          2202 non-null 

### Собираем таблицу по данным из таблиц Состояние онко помощи (контингент).

In [27]:
#читаем первую таблицу, чтобы было к чему пришивать остальные
data_sop_population = pd.read_excel(sop_population[0], header=[1,2])

#создаём колонки с годом, заболеванием, кодом заболевания, номером таблицы
new_cols = sop_new_cols(sop_population[0])
for key, value in new_cols.items():
    data_sop_population[key] = value

In [28]:
#читаем и конкатенируем все остальные из списка sop_population
for file in sop_population[1:]:
    df = pd.read_excel(file, header=[1,2])
    new_cols = sop_new_cols(file)
    for key, value in new_cols.items():
        df[key] = value
    data_sop_population = pd.concat([data_sop_population, df])

In [29]:
#убираем аггрегированные строки (по стране и по ФО)
data_sop_population = remove_aggregated_lines(data_sop_population)

In [30]:
#добавляем колонку с федеральным округом для каждой записи
data_sop_population['federal_district'] = data_sop_population.iloc[:, 0].map(fo_dict)

In [31]:
#поправляем колонки
data_sop_population.columns = column_names_change(data_sop_population)

In [32]:
#подправляем типы
data_sop_population['year'] = data_sop_population['year'].astype(int)
data_sop_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 14 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   республика, край, область                                   2380 non-null   object 
 1   взято на учет больных с впервые в жизни уст. диагнозом зно  2380 non-null   int64  
 2   в т.ч. выявлены активно, %                                  2380 non-null   float64
 3   находились на учете на конец года абсолютное число          2380 non-null   int64  
 4   находились на учете на конец года на 100 тыс. населения     2380 non-null   float64
 5   из них 5 лет и более абсолютное число                       2380 non-null   int64  
 6   из них 5 лет и более % от сост. на учете                    2380 non-null   float64
 7   индекс накопления контингентов                              2380 non-null   float64
 8 

### Собираем таблицу по данным из таблиц Злокачественные новообразования (заболеваемость и смертность)

In [33]:
#функция, которая формирует таблицу: разрезает вдоль на три части по полу и конкатенирует.
#таблицы заболеваний, характерных которые для одного пола, не трогает
#добавляет колонки: пол, год, название болезни, код болезни, номер таблицы
def process_table(table):

  workbook = openpyxl.load_workbook(table, read_only=True)
  num_columns = workbook.worksheets[0].max_column

  if num_columns > 10:
    #читаем табилицу
    df = pd.read_excel(table, header=[5, 6])
    #разрезаем вдоль по полу (все, мужчины, женщины), складываем по-отдельности в список
    lst = []
    for i in range(0, 3):
      lst.append(df.iloc[:, [0] + [4*i + j + 1 for j in range(4)]])

    #проходим по списку из разрезанных таблиц
    for i, item in enumerate(lst):
      #приводим в порядок названия столбцов
      item.columns = column_names_change(item)

      #добавляем столбец пола
      #читаем, к какому полу относится кусок таблицы, добавляем в новый столбец
      workbook = openpyxl.load_workbook(table, read_only=True)
      worksheet = workbook.worksheets[0]
      item['sex'] = worksheet.cell(row=5, column=4*i + 2).value

    #конкатенируем в одну таблицу
    df = pd.concat([lst[0], lst[1], lst[2]])

  else:
    #читаем таблицу
    df = pd.read_excel(table, header=[4, 5])
    #приводим в порядок названия столбцов
    df.columns = column_names_change(df)
    #добавляем столбец пола
    df['sex'] = None

  #создаём колонки с годом, заболеванием, кодом заболевания, номером таблицы
  new_cols = zno_new_cols(table)
  for key, value in new_cols.items():
      df[key] = value

  return df

In [34]:
#собираем первую таблицу
data_zno = process_table(zno_incidence[0])
data_zno['category'] = 'заболеваемость'

In [35]:
#добавляем остальные
for i, lst in enumerate([zno_incidence[1:], zno_mortality]):
  for file in tqdm(lst):
    data_zno = pd.concat([data_zno, process_table(file)])
  if i == 0:
    data_zno['category'] = data_zno['category'].fillna('заболеваемость')
  else:
    data_zno['category'] = data_zno['category'].fillna('смертность')

100%|██████████| 35/35 [00:59<00:00,  1.69s/it]


In [36]:
#убираем аггрегированные строки
data_zno = remove_aggregated_lines(data_zno)

In [37]:
#добавляем колонку с федеральным округом для каждой записи
data_zno['federal_district'] = data_zno.iloc[:, 0].str.strip().map(fo_dict)

In [38]:
#подправляем типы
data_zno['year'] = data_zno['year'].astype(int)

In [39]:
#смотрим, в каких таблицах не было указаний на пол
data_zno.query('sex.isna()')['localization'].unique()

array(['Вульва', 'Влагалище', 'Половой член', 'Тело матки', 'Яичник',
       'Шейка матки', 'Яичко', 'Предстательная железа', 'Плацента',
       'Другие новообразования матки', 'Другие женские половые органы',
       'Другие мужские половые органы'], dtype=object)

In [40]:
#собираем словарик полов для таблиц, в которых не было указаний на пол
sex_dictionary = {"Женщины": ["Влагалище", "Вульва", "Яичник", "Шейка матки", "Тело матки", "Плацента", "Другие женские половые органы", "Другие новообразования матки"]
                  , "Мужчины": ["Половой член", "Яичко", "Предстательная железа", "Другие мужские половые органы"]}

In [41]:
#применяем его к датасету
for key, value in sex_dictionary.items():

    condition = data_zno['localization'].isin(value)
    data_zno['sex'] = data_zno.apply(
        lambda row: key if pd.isna(row['sex']) and condition[row.name] else row['sex'],
        axis=1
    )

In [42]:
data_zno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18615 entries, 0 to 18614
Data columns (total 12 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   республика, край, область                           18615 non-null  object
 1   абсолютное число                                    18615 non-null  int64 
 2   показатель на 100 тыс. населения "грубый"           18615 non-null  object
 3   показатель на 100 тыс. населения стандартизованный  18615 non-null  object
 4   показатель на 100 тыс. населения ошибка             18615 non-null  object
 5   sex                                                 18615 non-null  object
 6   year                                                18615 non-null  int64 
 7   localization                                        18615 non-null  object
 8   icd_code                                            18615 non-null  object
 9   table 

### Собираем таблицу по данным из таблиц Состояние онко помощи (вторая вкладка).

In [43]:
data_sop_second = pd.read_excel(sop_population[0], sheet_name=1, header=[1, 2, 3])

new_cols = sop_new_cols(sop_population[0])
for key, value in new_cols.items():
    data_sop_second[key] = value

data_sop_second.columns = column_names_change(data_sop_second)

In [44]:
for file in tqdm(sop_population[1:]):
    try:
      df = pd.read_excel(file, sheet_name=1, header=[1, 2, 3])
    except Exception:
      continue
    new_cols = sop_new_cols(file)
    for key, value in new_cols.items():
        df[key] = value
    df.columns = data_sop_second.columns
    data_sop_second = pd.concat([data_sop_second, df])

100%|██████████| 27/27 [00:34<00:00,  1.27s/it]


In [45]:
#удаляем строки, которые получились из номеров страниц
data_sop_second = data_sop_second.dropna()

In [46]:
#убираем аггрегированные строки
data_sop_second = remove_aggregated_lines(data_sop_second)

In [47]:
#добавляем колонку с федеральным округом для каждой записи
data_sop_second['federal_district'] = data_sop_second.iloc[:, 0].str.strip().map(fo_dict)

In [48]:
#подправляем типы
data_sop_second['year'] = data_sop_second['year'].astype(int)
data_sop_second.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2206 entries, 0 to 2205
Data columns (total 14 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   республика, край, область                              2206 non-null   object 
 1   зарегистрировано зно (без учтенных посмертно)          2206 non-null   float64
 2   из них диагноз подтвержден морфологически, %           2206 non-null   float64
 3   из них имели стадию заболевания, % i                   2206 non-null   object 
 4   из них имели стадию заболевания, % ii                  2206 non-null   object 
 5   из них имели стадию заболевания, % iii                 2206 non-null   object 
 6   из них имели стадию заболевания, % iv                  2206 non-null   object 
 7   из них имели стадию заболевания, % не установлена      2206 non-null   object 
 8   летальность на первом году с момента уст. диагно

### Загружаем данные в гугл-шитс

In [49]:
#подсоединяемся
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(f"{load_path}/onco-file-processor-dc07dc7f89c4.json", scope)
client = gspread.authorize(creds)

In [51]:
#создаём словарь таблиц и названий
final_list = {"Сотояние_онко_помощи_лечение": data_sop_treatment
              , "Состояние_онко_помощи_контингент": data_sop_population
              , "Состояние_онко_помощи_контингент_вкл2": data_sop_second
              , "Злокачественные_новообразования_заболеваемость_и_смертность": data_zno}

In [52]:
for key, value in final_list.items():
  spreadsheet = client.create(key)
  worksheet = spreadsheet.sheet1
  worksheet.update([value.columns.values.tolist()] + value.values.tolist())
  spreadsheet.share("", perm_type="anyone", role="reader")
  print(f'{key}: {spreadsheet.url}')

Сотояние_онко_помощи_лечение: https://docs.google.com/spreadsheets/d/1GlkbPKtB7-iS8Nqioe9FVqxgmbxf6KuS-hgcgdJb6MQ
Состояние_онко_помощи_контингент: https://docs.google.com/spreadsheets/d/10hXq4zzDZcGI7Y_VSIYHxmA8x891hEu6zDLMSaMDfiI
Состояние_онко_помощи_контингент_вкл2: https://docs.google.com/spreadsheets/d/1tnyQOEBTJgH4IKya754y3wpGaR7kovry-26NKc302GQ
Злокачественные_новообразования_заболеваемость_и_смертность: https://docs.google.com/spreadsheets/d/1AsSU5bg51t62I6ayMzxwPaqyc4EXeTz3fgc_7klaa0E
